The entirety of this notebook originates from the DataQuest tutorial and has been reformatted to fit this Jupyter Notebook. For the original tutorial, please see https://www.dataquest.io/blog/python-api-tutorial/

Tutorial author: Vik Paruchuri
(reformatted by Nick Beaudoin)

## Type of requests
There are many different types of requests. The most commonly used one, a GET request, is used to retrieve data.

We can use a simple GET request to retrieve information from the OpenNotify API.

OpenNotify has several API endpoints. An endpoint is a server route that is used to retrieve different data from the API. For example, the /comments endpoint on the Reddit API might retrieve information about comments, whereas the /users endpoint might retrieve data about users. To access them, you would add the endpoint to the base url of the API.

The first endpoint we’ll look at on OpenNotify is the iss-now.json endpoint. This endpoint gets the current latitude and longitude of the International Space Station. As you can see, retrieving this data isn’t a great fit for a dataset, because it involves some calculation on the server, and changes quickly.

The base url for the OpenNotify API is http://api.open-notify.org, so we’ll add this to the beginning of all of our endpoints.

In [1]:
import requests

# Make a get request to get the latest position of the international space station from the opennotify api.
response = requests.get("http://api.open-notify.org/iss-now.json")

# Print the status code of the response.
print(response.status_code)

200


The request we just made had a status code of 200. Status codes are returned with every request that is made to a web server. Status codes indicate information about what happened with a request. Here are some codes that are relevant to GET requests:

 - 200 — everything went okay, and the result has been returned (if any)
 - 301 — the server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.
 - 401 — the server thinks you’re not authenticated. This happens when you don’t send the right credentials to access an API (we’ll talk about authentication in a later post).
 - 400 — the server thinks you made a bad request. This can happen when you don’t send along the right data, among other things.
 - 403 — the resource you’re trying to access is forbidden — you don’t have the right permissions to see it.
 - 404 — the resource you tried to access wasn’t found on the server.

We’ll now make a GET request to http://api.open-notify.org/iss-pass, an endpoint that doesn’t exist, per the API documentation.

In [2]:
response = requests.get("http://api.open-notify.org/iss-pass")
print(response.status_code)

404


## Hitting the right endpoint
iss-pass wasn’t a valid endpoint, so we got a 404 status code in response. We forgot to add .json at the end, as the API documentation states.

We’ll now make a GET request to http://api.open-notify.org/iss-pass.json.

In [3]:
response = requests.get("http://api.open-notify.org/iss-pass.json")
print(response.status_code)

400


## Query parameters
You’ll see that in the last example, we got a 400 status code, which indicates a bad request. If you look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.

The ISS Pass endpoint returns when the ISS will next pass over a given location on earth. In order to compute this, we need to pass the coordinates of the location to the API. We do this by passing two parameters — latitude and longitude.

We can do this by adding an optional keyword argument, params, to our request. In this case, there are two parameters we need to pass:

lat — The latitude of the location we want.
lon — The longitude of the location we want.
We can make a dictionary with these parameters, and then pass them into the requests.get function.

We can also do the same thing directly by adding the query parameters to the url, like this: http://api.open-notify.org/iss-pass.json?lat=40.71&lon=-74.

It’s almost always preferable to setup the parameters as a dictionary, because requests takes care of some things that come up, like properly formatting the query parameters.

We’ll make a request using the coordinates of New York City, and see what response we get.

In [4]:
# Set up the parameters we want to pass to the API.
# This is the latitude and longitude of New York City.
parameters = {"lat": 40.71, "lon": -74}
print(parameters)

{'lat': 40.71, 'lon': -74}


In [5]:
# Make a get request with the parameters.
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

In [6]:
# Print the content of the response (the data the server returned)
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1592585263, \n    "latitude": 40.71, \n    "longitude": -74.0, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 646, \n      "risetime": 1592589291\n    }, \n    {\n      "duration": 601, \n      "risetime": 1592595109\n    }, \n    {\n      "duration": 573, \n      "risetime": 1592649497\n    }, \n    {\n      "duration": 650, \n      "risetime": 1592655247\n    }, \n    {\n      "duration": 585, \n      "risetime": 1592661119\n    }\n  ]\n}\n'


## Working with JSON data

You may have noticed that the content of the response earlier was a string (although it was shown as a `bytes` object, we can easily convert the content to a string using `response.content.decode("utf-8"))`.

Strings are the way that we pass information back and forth to APIs, but it’s hard to get the information we want out of them. How do we know how to decode the string that we get back and work with it in Python? How do we figure out the `altitude` of the ISS from the string response?

Luckily, there’s a format called _JavaScript Object Notation (JSON)_. JSON is a way to encode data structures like lists and dictionaries to strings that ensures that they are easily readable by machines. JSON is the primary format in which data is passed back and forth to APIs, and most API servers will send their responses in JSON format.

Python has great JSON support, with the `json` package. The `json` package is part of the standard library, so we don’t have to install anything to use it. We can both convert lists and dictionaries to JSON, and convert strings to lists and dictionaries. In the case of our ISS Pass data, it is a dictionary encoded to a string in JSON format.

The json library has two main methods:

 - dumps — Takes in a Python object, and converts it to a string.
 - loads — Takes a JSON string, and converts it to a Python object.

In [7]:
# Make a list of fast food chains.
best_food_chains = ["Taco Bell", "Shake Shack", "Chipotle"]

In [8]:
# This is a list.
print(type(best_food_chains))

<class 'list'>


In [9]:
# Import the json library
import json

In [10]:
# Use json.dumps to convert best_food_chains to a string.
best_food_chains_string = json.dumps(best_food_chains)

In [11]:
# We've successfully converted our list to a string.
print(type(best_food_chains_string))

<class 'str'>


In [12]:
# Convert best_food_chains_string back into a list
print(type(json.loads(best_food_chains_string)))

<class 'list'>


In [13]:
# Make a dictionary
fast_food_franchise = {
    "Subway": 24722,
    "McDonalds": 14098,
    "Starbucks": 10821,
    "Pizza Hut": 7600
}

In [14]:
# We can also dump a dictionary to a string and load it.
fast_food_franchise_string = json.dumps(fast_food_franchise)

print(type(fast_food_franchise_string))

<class 'str'>


## Getting JSON from an API request
You can get the content of a response as a python object by using the .json() method on the response.

In [15]:
# Make the same request we did earlier, but with the coordinates of San Francisco instead.
parameters = {"lat": 37.78, "lon": -122.41}

In [16]:
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

In [17]:
response

<Response [200]>

In [18]:
# Get the response data as a python object. Verify that it's a dictionary.
data = response.json()

In [19]:
print(type(data))

print('-------')

print(data)

<class 'dict'>
-------
{'message': 'success', 'request': {'altitude': 100, 'datetime': 1592585270, 'latitude': 37.78, 'longitude': -122.41, 'passes': 5}, 'response': [{'duration': 508, 'risetime': 1592588765}, {'duration': 510, 'risetime': 1592594653}, {'duration': 623, 'risetime': 1592600458}, {'duration': 630, 'risetime': 1592606263}, {'duration': 105, 'risetime': 1592612281}]}


## Content type
The server doesn’t just send a status code and the data when it generates a response. It also sends metadata containing information on how the data was generated and how to decode it. This is stored in the _response headers_. In Python, we can access this with the `headers` property of a response object.

The headers will be shown as a dictionary. Within the headers, `content-type` is the most important key for now. It tells us the format of the response, and how to decode it. For the OpenNotify API, the format is JSON, which is why we could decode it with the json package earlier.

In [20]:
# Headers is a dictionary
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Fri, 19 Jun 2020 16:47:50 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


In [21]:
# Get the content-type from the dictionary.
print(response.headers["content-type"])

application/json


## Finding the number of people in space
OpenNotify has one more API endpoint, astros.json. It tells you how many people are currently in space. The format of the responses can be found [here] (http://open-notify.org/Open-Notify-API/People-In-Space/).

In [22]:
# Get the response from the API endpoint.
import requests

response = requests.get("http://api.open-notify.org/astros.json")
data = response.json()

In [23]:
# 9 people are currently in space.
print(data)

{'message': 'success', 'number': 5, 'people': [{'craft': 'ISS', 'name': 'Chris Cassidy'}, {'craft': 'ISS', 'name': 'Anatoly Ivanishin'}, {'craft': 'ISS', 'name': 'Ivan Vagner'}, {'craft': 'ISS', 'name': 'Doug Hurley'}, {'craft': 'ISS', 'name': 'Bob Behnken'}]}


In [24]:
import pandas as pd
from pandas.io.json import json_normalize


In [25]:
df = pd.DataFrame.from_dict(data, orient='columns')

In [26]:
df

,message,number,people
0,success,5,"{'craft': 'ISS', 'name': 'Chris Cassidy'}"
1,success,5,"{'craft': 'ISS', 'name': 'Anatoly Ivanishin'}"
2,success,5,"{'craft': 'ISS', 'name': 'Ivan Vagner'}"
3,success,5,"{'craft': 'ISS', 'name': 'Doug Hurley'}"
4,success,5,"{'craft': 'ISS', 'name': 'Bob Behnken'}"


In [27]:
pd.concat([df.drop(['people'], axis=1), df['people'].apply(pd.Series)], axis=1)

,message,number,craft,name
0,success,5,ISS,Chris Cassidy
1,success,5,ISS,Anatoly Ivanishin
2,success,5,ISS,Ivan Vagner
3,success,5,ISS,Doug Hurley
4,success,5,ISS,Bob Behnken


In [28]:
df.to_csv('ISS.csv')